In [1]:
import import_ipynb
import jpype as jp
import pandas as pd
from typing import List
from jpype import (JClass, JString, getDefaultJVMPath, java, shutdownJVM,
                   startJVM)
import re
import Module1
from Module1 import letter_values
# import find_pos

import string
## Zemberek: Finding POS Tag Example
# Documentation: https://github.com/ahmetaa/zemberek-nlp/tree/master/morphology
# Java Code Example: https://github.com/ahmetaa/zemberek-nlp/blob/master/examples/src/main/java/zemberek/examples/morphology/FindPOS.java

# Relative path to Zemberek .jar
ZEMBEREK_PATH = '/home/busra/zemberek-full.jar'

# Start the JVM
jp.startJVM(jp.getDefaultJVMPath(), '-ea', '-Djava.class.path=%s' % (ZEMBEREK_PATH))

# Import the required Java classes
TurkishMorphology = jp.JClass('zemberek.morphology.TurkishMorphology')

# Instantiating the morphology class with the default RootLexicon
morphology = TurkishMorphology.createWithDefaults()
positive_negatives: List[JString] = [JString(''), JString('Neg')]
times: List[JString] = [
    'Imp', 'Aor', 'Past', 'Prog1', 'Prog2', 'Narr', 'Fut'  # time suffixes
]
people: List[JString] = [
    'A1sg', 'A2sg', 'A3sg', 'A1pl', 'A2pl', 'A3pl'  # person suffixes
]


synomym_noun_dict = {}
synomym_verb_dict = {}
sentences_list = list()
sentences_new_list = list()
sentences_and_values_dict = {}
def create_synonym_nouns_dictionary():
    file = "/home/busra/PycharmProjects/Piton/myenv/Zemberek-Python-Examples-master/examples/morphology/stem-and-lemmatize/synonyms.txt"# give synonym txt path name
    with open(file, 'r') as f:
        for line in f:
            line = line.split()
            try:
                try:
                    synomym_noun_dict[line[0]] = line[1]
                except IndexError:
                    continue
            except KeyError:
                continue
                

def create_sentences_list():
    file = "/home/busra/PycharmProjects/Piton/myenv/Zemberek-Python-Examples-master/examples/morphology/stem-and-lemmatize/sentences.txt"# give sentence txt path name
    with open(file, 'r') as f:
        for line in f:
            line = line.split()
            sentences_list.append(line)


def create_synonym_verbs_dictionary():
    file = "/home/busra/PycharmProjects/Piton/myenv/Zemberek-Python-Examples-master/examples/morphology/stem-and-lemmatize/closeMeaningVerbs"# give closeMeaningVerbs txt path name
    with open(file, 'r') as f:
        for line in f:
            line = line.split(":")
            synomym_verb_dict[line[0]] = line[1]


def analize(sentence):
    if not sentence:
        return 0
    sentence2 = " ".join(sentence)
    analysis = morphology.analyzeAndDisambiguate(sentence2).bestAnalysis()
    new_sentence = list()
    verb = ""
    sy_verb = ""
    verb_to_conjugate = ""
    if "Verb" not in str(analysis): #cümlede verb yoksa cümleyi atla
        return 3
    for i, a in enumerate(analysis):
        # print('Analysis %d: %s' % (i+1, a))#a kullanılcak
        # print('Primary POS %d: %s' % (i+1, a.getPos()))

        if "Verb" in str(a) and "Noun" not in str(a) and "Adj" not in str(a) and "Adv" not in str(a):
            verb_to_conjugate = str(a)
            word_temp = str(a)[1:]
            word_temp = word_temp.split(":")
            word = word_temp[0]
            verb = word
            if word in synomym_verb_dict:
                sy_verb = synomym_verb_dict.get(word)
                break
    if not verb:
        return 2
    if "mek" in verb:
        verb2 = verb.split("mek")
        verb = verb2[0]
    elif "mak" in verb:
        verb2 = verb.split("mak")
        verb = verb2[0]
    if sy_verb:
        for i, word in enumerate(sentence):
            if verb in word:
                ret_val = conjugate_verb(verb_to_conjugate,sy_verb) #çekimleme olayı burada yapılacak
                if ret_val == None:
                    break
                if "mek" in ret_val:
                    ret_val=re.sub("mek", "", ret_val)
                elif "mak" in ret_val:
                    ret_val=re.sub("mak", "", ret_val)
                sentence[i] = ret_val
                break
    return sentence


def conjugate_verb(verb, sy_verb):

    morphology: TurkishMorphology = (
        TurkishMorphology.builder().setLexicon(sy_verb).disableCache().build()
    )

    stem: str = sy_verb

    for pos_neg in positive_negatives:  # add positive and negative suffiex to chosen words
        for time in times:  # add time suffixes to chosen closeMeaningVerbs
            for person in people:  # add person suffixes to chosen closeMeaningVerbs

                seq: java.util.ArrayList = java.util.ArrayList()  # put this all suffixes to seq
                if pos_neg:
                    seq.add(JString(pos_neg))
                if time:
                    seq.add(JString(time))
                if person == 'A2sg' or person == 'A1sg' or person == 'A3sg' or person == 'A1pl' or person == 'A2pl' or person == 'A3pl':
                    seq.add(JString(person))

                results = list(morphology.getWordGenerator().generate(
                    # generate new conjugate words from verb stems with suffixes in seq
                    JString(stem),
                    seq
                ))
                if not results:
                    continue
                for morph in seq:

                    if 'A1sg' in str(verb):
                        if 'A1sg' in str(morph):
                            control = control_tenses(verb, time, results)
                            if control:
                                return control


                    elif 'A2sg' in str(verb):
                        if 'A2sg' in str(morph):
                            control = control_tenses(verb, time, results)
                            if control:
                                return control


                    elif 'A3sg' in str(verb):
                        if 'A3sg' in str(morph):
                            print(str())
                            control = control_tenses(verb, time, results)
                            if control:
                                return control
                            # for result in results:
                            #     print(str(result.surface))

                    elif 'A1pl' in str(verb):
                        if 'A1pl' in str(morph):
                            control = control_tenses(verb, time, results)
                            if control:
                                return control


                    elif 'A2pl' in str( verb):
                        if 'A2pl' in str(morph):
                            control = control_tenses(verb, time, results)
                            if control:
                                return control


                    elif 'A3pl' in str(verb):
                        if 'A3pl' in str(morph):
                            control = control_tenses(verb, time, results)
                            if control:
                                return control


def control_tenses(verb, time, results):
    # 'Imp', 'Aor', 'Past', 'Prog1', 'Prog2', 'Narr', 'Fut'

    if 'Imp' in str(verb) and 'Imp' in str(time):
        for result in results:
            return str(result.surface)
    if 'Aor' in str(verb) and 'Aor' in str(time):
        for result in results:
            return str(result.surface)
    if 'Past' in str(verb) and 'Past' in str(time):
        for result in results:
            return str(result.surface)
    if 'Prog1' in str(verb) and 'Prog1' in str(time):
        for result in results:
            return str(result.surface)
    if 'Prog2' in str(verb) and 'Prog2' in str(time):
        for result in results:
            return str(result.surface)

    if 'Narr' in str(verb) and 'Narr' in str(time):
        for result in results:
            return str(result.surface)

    if 'Fut' in str(verb) and 'Fut' in str(time):
        for result in results:
            return str(result.surface)


def searh_word_in_dict(all_files): #file a sentenceları yazma
    for i in synomym_noun_dict:
        for file in all_files:
            with open(file, 'r') as f:
                for line in f:
                    line3 = line.split(".")
                    for j in line3:
                        line2 = j.split()
                        if i in line2:
                            print(j)
    allFilesPath = Module1.get_list_of_files("/home/busra/1150haber")#corpus that have datas
    searh_word_in_dict(allFilesPath)


def change_words_with_synonym():
    for sentence in sentences_list:
        for i,word in enumerate(sentence):
            if word in synomym_noun_dict:
                synonym_word = synomym_noun_dict.get(word)
                sentence[i] = synonym_word


def create_new_sentences():
    for i, sentence in enumerate(sentences_list):
        neew_sentence = analize(sentence)
        if neew_sentence == 0 or neew_sentence == 2 or neew_sentence == 3:
            continue
        sentences_new_list.append(neew_sentence)


def write_to_the_file():
    file = "/home/busra/PycharmProjects/Piton/myenv/Zemberek-Python-Examples-master/examples/morphology/stem-and-lemmatize/new_sentences"# give new_sentences txt path name
    with open(file, 'w') as f:
        for line in sentences_new_list:
            if line is None:
                continue
            line2 = " ".join(line) + "\n"
            count_of_sentence = counter(line2)
            sentences_and_values_dict[count_of_sentence] = line2
            print(line2)
            f.write(line2)
            


def counter(sentence):
    sentence2 = sentence.split()
    sentence_counter = 0
    for word in sentence2:
        if letter_values(word) == 0:
            continue
        count, word2 = letter_values(word)
        sentence_counter = sentence_counter + int(count)
        
    return sentence_counter

#wait for a two minute for running ...
if __name__ == '__main__':
    print("wait for a two minute for running ...")
    create_synonym_verbs_dictionary()
    create_sentences_list()
    create_synonym_nouns_dictionary()
    change_words_with_synonym()
    create_new_sentences()
    write_to_the_file()
    value = int(input("Please enter the value: "))
    print(sentences_and_values_dict.get(value))
    while(sentences_and_values_dict.get(value) == None):
        value = int(input("Please enter the value: "))
        print(sentences_and_values_dict.get(value))

importing Jupyter notebook from Module1.ipynb


/home/busra/PycharmProjects/Piton/myenv/lib/python3.6/site-packages/jpype/_core.py:218: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


wait for a two minute for running ...





































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































Velioğlu, bu işe fazla iyi hazırlanan Yapı Kredi Emeklilik'in pazarlamaya önem verdiğini ve 450 kişilik doğrudan satış ekibi bulunduğunu belirterek, 'Ektiğimizi biçiyoruz

Büyüklüğü göz önüne alındığında, bugün için kısa sürede özelleştirilmesi fazla mümkün değil

﻿GS'nin kârına en fazla bankalar sevinecek

Roche ilaç şirketinin eski çalışan Veysi Mungan, SSK ilaç alım ihalelerinde "Neorecormon" isimli ilacın piyasa değerinin fazla üzerinde satılarak SSK'nın Aralık 2003 fiyatlarıyla 10 milyon doların üzerinde zarara uğratıldığı iddia etmişti

Elit bayanların alışveriş yapacağı fazla modern bir mağaza olacak

Bir fazla yabancı dev alıcı da eş refleksleri gösteriyor

Başlıkları baktım ve sordum; "Nedir bizleri endişelendiren? Bizler acep fazla mu paranoyak adamlarız?"

2000 yılında oynan3an oyuna karşı bilinçlenirken, işin ekonomiden ulaşacak değişim kısmına karşı tedbir aldık, fakat başka tarafı fazla çok ihmal ettik

Böyle bir merkezi bürokrasinin yanında mahalli yönetimlere geniş yet


Uluslararası olmamış petrol fiyatları, 45 dolar düzeyine erişti

ABD olmamış petrolü 1

Meksika Körfezi'nde yaşanan kasırga, Rus petrol şirketi Yukos'un krizde olması gibi etkenlerinde piyasayı menfi etkilediğine değinen uzmanlar, ABD olmamış petrolünün fazla spekülatif bir piyasayada işlem gördüğünü anımsatarak, duygulu dengeler üzerine toplamış piyasanın en küçük bir kaygı de bile menfi etkilendiğini sözlerine ekledi

47 dolar olan olmamış petrol varil fiyatı yüzde 270 artarak 38

GEÇEN haftayı tarihi zirvelerinde kapatan olmamış petrol fiyatları, yeni haftaya da yeni rekorlarla girdi

New York Ticaret Borsası'nda işlem gören hafif türü olmamış petrolün varil fiyatı, 21 yıllık borsa tarihinin en yüksek değeri olan 46

Son gerçekleşen yükselişlerle birlikte olmamış petrol fiyatları New York Ticaret Borsası'nda, 1 sene öncesine göre yüzde 50'nin üzerinde kıymet edinmiş oldu

Venezuella devlet petrol şirketi PDVSA Başkanı Ali Rodriguez olmamış petrol fiyatlarının bu hafta içinde 47-48 

UEFA Asbaşkanı, "Bu vaka beni gururlandırdı" söyleti

FENERBAHÇE’NİN Dereağzı tesislerinde oynanan PAF maçında vaka gitti

Bir de 20’li dakikalarda enteresan bir vaka yaşadı

Gaziantepspor Başkanı Celal Doğan, "Küfüre, kavgaya hayır" söyleti taraftarlar tekrar de vaka çıkardı

Anne, şaşırtıcı vaka sonrası, "Maça bilet aldığımızda henüz çocuğumuz doğmamıştı bile" söyleti

Mesela Hasan Şaş "piyasa şartlarında" hak ettiği parayı alabilmek için direniyorsa ve bu vaka Milli Takım teknik direktörünün Federasyon Başkanı ile "dolar kuru" yüzünden birbirine girdiği bir ülkede yapıyorsa, Hasan Şaş ne kadar suçlanabilirdi?

İngiltere'de yayımlanan Times gazetesinin manşetine taşıdığı habere göre Abromoviç'in adının geçtiği bu vaka İsviçreli yetkililerce soruşturulmaya başlandı

6 milyar dolarlık şahsi servetiyle dünyanın en varlıklı 25'inci kişisi seçilen 37 yaşındaki Abramovich'in adı modern zamanların en büyük finansal bilmecesi olarak adlandırılan bu vaka için İsviçreli yetkililer tarafından h


Bir yumruk büyüklüğündeki yapay kalp, dakikada yaklaşık 8 litre kan pompalıyor

Bugünkü aşamada, cisimlerin gösterilmesi ile görme özürlü denek tarafından algılanması arasında 10 saniye kadar vakit geçtiğini belirten yararlandılar yüksek teknolojinin bu alanda hız ve kabiliyet getireceğine inandıklarını anlattılar ve insan organizmasının, keşfedilen sistemi kabullenmesinin önemi üzerinde durarak, çalışmaların önünün açık olduğunu kaydettiler

İNGİLTERE’NİN Aston Villa takımında sıkıntılı günler geçiren, hakkında devamlı spekülasyonlar yapılan Milli futbolcu Alpay Özalan, Abu Dhabi’de Milliyet’in sorularını yanıtladı, yaşadığı olayların perde arkasını yararlandı

Ulusal kimliğimizle devamlı ümit yorumları yaptık

BBC’YE konuşan Ukraynalı menajer, "Fenerli yöneticiler bizi saat başı arıyorlar ve devamlı tekliflerini bildiriyorlar

VLADİMİR, Fenerbahçe’deki ilk maçında sahada bulunduğu süre içinde devamlı top almak için çalıştı

Açık sözlülüğüyle meşhur Rus yıldız, 1999 - 2000 sezonunda 


Yeşil kartının vizesinin bittiğini, bu şekilde hastanenin acil servisine gelip gelemeyeceğini soran Akdağ, acil durumlarda kendisine vizesi bitmesine karşın muavin olunabileceği karşılığını aldı

Nature dergisindeki habere göre, Austin Üniversitesi'nde görevli ilim adamı Adam Heller, pilin deri altına yerleştirilerek hastanın sıhhat durumunu kontrol etmeye muavin olacağını anlattı

Bunu daha küçük yaparak köprücük kemiğinin altını kesip buradan girerek, göğsü açmadan kemiğin altına muavin kalbi yerleştireceğim

Bilim adamları, bu eldivene eş muavin protezlerin ileride bacak gibi başka uzuvlar için de geliştirmeyi planlıyor

Işığı dağıtan polimerler de ten kusurlarını kamufle ediyor ve ten tonunun dengelenmesine muavin oluyor

Üstelik, yeni teknolojiler, devamlı otomobil kullanımı, televizyon ve bilgisayar kullanımı da, bu konudaki problemlerimizi halletmemize pek muavin olmuyor

Avustralya hükümetinin, bir tabanca veya kurutma makinesine benzeyen tarayıcı aletin gelişmesine muavin ola

Please enter the value: 495
Beşiktaş’ta inanılmaz fizik kuvvet var

